LAB2 - IE212.Q11.1, 22521121 - Lê Thiên Phúc

In [1]:
!pip install pyspark

In [3]:
from pyspark import SparkContext
import datetime

try:
    sc.stop()
except:
    pass
sc = SparkContext(appName="IE212_Lab2")

print("SparkContext đã chạy")

SparkContext đã chạy


In [4]:
movies_rdd = sc.textFile("movies.txt") \
    .map(lambda line: line.split(',')) \
    .map(lambda fields: (int(fields[0]), fields[1]))

movies_genres_rdd = sc.textFile("movies.txt") \
    .map(lambda line: line.split(',')) \
    .map(lambda fields: (int(fields[0]), fields[2].split('|')))

ratings1_rdd = sc.textFile("ratings_1.txt")
ratings2_rdd = sc.textFile("ratings_2.txt")
all_ratings_rdd = ratings1_rdd.union(ratings2_rdd) \
    .map(lambda line: line.split(',')) \
    .map(lambda fields: (int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3])))

users_rdd = sc.textFile("users.txt") \
    .map(lambda line: line.split(',')) \
    .map(lambda fields: (int(fields[0]), (fields[1], int(fields[2]), int(fields[3]))))

occupation_rdd = sc.textFile("occupation.txt") \
    .map(lambda line: line.split(',')) \
    .map(lambda fields: (int(fields[0]), fields[1]))

# Hiển thị 5 dòng đầu để check
print("\nMovies (ID, Title):")
for row in movies_rdd.take(5): print(row)

print("\nRatings (UserID, MovieID, Rating, Timestamp):")
for row in all_ratings_rdd.take(5): print(row)

print("\nUsers (UserID, (Gender, Age, OccID)):")
for row in users_rdd.take(5): print(row)

print("\nOccupation (OccID, Name):")
for row in occupation_rdd.take(5): print(row)


Movies (ID, Title):
(1001, 'The Godfather (1972)')
(1002, 'The Shawshank Redemption (1994)')
(1003, "Schindler's List (1993)")
(1004, 'Raging Bull (1980)')
(1005, 'Casablanca (1942)')

Ratings (UserID, MovieID, Rating, Timestamp):
(7, 1020, 4.5, 1577836800)
(23, 1015, 3.5, 1577923200)
(45, 1030, 4.0, 1578009600)
(12, 1047, 3.0, 1578096000)
(38, 1012, 4.5, 1578182400)

Users (UserID, (Gender, Age, OccID)):
(1, ('M', 28, 3))
(2, ('F', 35, 7))
(3, ('M', 42, 2))
(4, ('F', 19, 10))
(5, ('M', 31, 1))

Occupation (OccID, Name):
(1, 'Programmer')
(2, 'Doctor')
(3, 'Engineer')
(4, 'Teacher')
(5, 'Lawyer')


 Bài 1: Tính ĐTB đánh giá và tổng số lượt đánh giá cho mỗi phim

In [8]:
movie_ratings_kv = all_ratings_rdd.map(lambda x: (x[1], (x[2], 1)))

movie_ratings_sum_count = movie_ratings_kv.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

movie_avg_ratings = movie_ratings_sum_count.mapValues(
    lambda v: (v[0] / v[1], v[1])
)

joined_rdd = movies_rdd.join(movie_avg_ratings)

formatted_rdd_b1 = joined_rdd.map(
    lambda x: f"{x[1][0]} AverageRating: {x[1][1][0]:.2f} (TotalRatings: {x[1][1][1]})"
)

all_results = formatted_rdd_b1.collect()

for line in all_results:
    print(line)

# Tìm phim có điểm trung bình cao nhất (tối thiểu 5 đánh giá)
filtered_for_max = movie_avg_ratings.filter(lambda x: x[1][1] >= 5)

max_rating_data = filtered_for_max.max(key=lambda x: x[1][0])
max_movie_id, (max_avg, max_count) = max_rating_data

max_movie_title = movies_rdd.lookup(max_movie_id)[0]

print(f"{max_movie_title} is the highest rated movie with an average rating of {max_avg:.2f} among movies with at least 5 ratings.")

E.T. the Extra-Terrestrial (1982) AverageRating: 3.67 (TotalRatings: 18)
Mad Max: Fury Road (2015) AverageRating: 3.47 (TotalRatings: 18)
Sunset Boulevard (1950) AverageRating: 4.36 (TotalRatings: 7)
The Lord of the Rings: The Return of the King (2003) AverageRating: 3.82 (TotalRatings: 11)
Lawrence of Arabia (1962) AverageRating: 3.44 (TotalRatings: 18)
Fight Club (1999) AverageRating: 3.50 (TotalRatings: 7)
Gladiator (2000) AverageRating: 3.61 (TotalRatings: 18)
The Social Network (2010) AverageRating: 3.86 (TotalRatings: 7)
Psycho (1960) AverageRating: 4.00 (TotalRatings: 2)
The Silence of the Lambs (1991) AverageRating: 3.14 (TotalRatings: 7)
The Godfather: Part II (1974) AverageRating: 4.00 (TotalRatings: 17)
The Terminator (1984) AverageRating: 4.06 (TotalRatings: 18)
The Lord of the Rings: The Fellowship of the Ring (2001) AverageRating: 3.89 (TotalRatings: 18)
No Country for Old Men (2007) AverageRating: 3.89 (TotalRatings: 18)
Sunset Boulevard (1950) is the highest rated movie

Bài 2: Phân tích đánh giá theo thể loại

In [9]:
movie_genre_flat_rdd = movies_genres_rdd.flatMapValues(lambda x: x)

movie_ratings = all_ratings_rdd.map(lambda x: (x[1], x[2]))

joined_genre_rdd = movie_genre_flat_rdd.join(movie_ratings)

genre_ratings_kv = joined_genre_rdd.map(lambda x: (x[1][0], (x[1][1], 1)))

genre_sum_count = genre_ratings_kv.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

genre_avg = genre_sum_count.mapValues(
    lambda v: (v[0] / v[1], v[1])
)

formatted_rdd_b2 = genre_avg.map(
    lambda x: f"{x[0]} - AverageRating: {x[1][0]:.2f} (TotalRatings: {x[1][1]})"
)

for line in formatted_rdd_b2.collect():
    print(line)

Sci-Fi - AverageRating: 3.73 (TotalRatings: 54)
Action - AverageRating: 3.71 (TotalRatings: 54)
Drama - AverageRating: 3.76 (TotalRatings: 128)
Family - AverageRating: 3.67 (TotalRatings: 18)
Biography - AverageRating: 3.56 (TotalRatings: 25)
Horror - AverageRating: 4.00 (TotalRatings: 2)
Fantasy - AverageRating: 3.86 (TotalRatings: 29)
Mystery - AverageRating: 4.00 (TotalRatings: 2)
Thriller - AverageRating: 3.70 (TotalRatings: 27)
Adventure - AverageRating: 3.63 (TotalRatings: 83)
Film-Noir - AverageRating: 4.36 (TotalRatings: 7)
Crime - AverageRating: 3.81 (TotalRatings: 42)


Bài 3: Phân tích đánh giá theo giới tính

In [11]:
user_gender_rdd = users_rdd.map(lambda x: (x[0], x[1][0]))

user_movie_rating_rdd = all_ratings_rdd.map(lambda x: (x[0], (x[1], x[2])))

joined_gender_data = user_movie_rating_rdd.join(user_gender_rdd)

key_gender_rdd = joined_gender_data.map(
    lambda x: ((x[1][0][0], x[1][1]), (x[1][0][1], 1))
)

gender_sum_count = key_gender_rdd.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

gender_avg = gender_sum_count.mapValues(lambda v: v[0] / v[1])

pivot_prep_rdd = gender_avg.map(lambda x: (x[0][0], (x[0][1], x[1])))

grouped_by_movie = pivot_prep_rdd.groupByKey()

def format_gender_ratings(ratings_list):
    male_avg = "NA"
    female_avg = "NA"
    for gender, avg in ratings_list:
        if gender == 'M':
            male_avg = f"{avg:.2f}"
        elif gender == 'F':
            female_avg = f"{avg:.2f}"
    return f"Male_Avg: {male_avg}, Female_Avg: {female_avg}"

gender_strings_rdd = grouped_by_movie.mapValues(format_gender_ratings)

final_gender_rdd = movies_rdd.join(gender_strings_rdd)

formatted_rdd_b3 = final_gender_rdd.map(
    lambda x: f"{x[1][0]} - {x[1][1]}"
)

all_results = formatted_rdd_b3.collect()

for line in all_results:
    print(line)

Gladiator (2000) - Male_Avg: 3.59, Female_Avg: 3.64
The Terminator (1984) - Male_Avg: 3.93, Female_Avg: 4.14
Lawrence of Arabia (1962) - Male_Avg: 3.55, Female_Avg: 3.31
Mad Max: Fury Road (2015) - Male_Avg: 4.00, Female_Avg: 3.32
No Country for Old Men (2007) - Male_Avg: 3.92, Female_Avg: 3.83
Psycho (1960) - Male_Avg: NA, Female_Avg: 4.00
E.T. the Extra-Terrestrial (1982) - Male_Avg: 3.81, Female_Avg: 3.55
Fight Club (1999) - Male_Avg: 3.50, Female_Avg: 3.50
The Godfather: Part II (1974) - Male_Avg: 4.06, Female_Avg: 3.94
The Lord of the Rings: The Fellowship of the Ring (2001) - Male_Avg: 4.00, Female_Avg: 3.80
The Silence of the Lambs (1991) - Male_Avg: 3.33, Female_Avg: 3.00
Sunset Boulevard (1950) - Male_Avg: 4.33, Female_Avg: 4.50
The Lord of the Rings: The Return of the King (2003) - Male_Avg: 3.75, Female_Avg: 3.90
The Social Network (2010) - Male_Avg: 4.00, Female_Avg: 3.67


Bài 4: Phân tích đánh giá theo nhóm tuổi

In [14]:
def get_age_group(age):
    if age <= 18: return "0-18"
    if age <= 35: return "18-35"
    if age <= 50: return "35-50"
    return "50+"

user_age_group_rdd = users_rdd.map(lambda x: (x[0], get_age_group(x[1][1])))

joined_age_data = user_movie_rating_rdd.join(user_age_group_rdd)

key_age_rdd = joined_age_data.map(
    lambda x: ((x[1][0][0], x[1][1]), (x[1][0][1], 1))
)

age_sum_count = key_age_rdd.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

age_avg = age_sum_count.mapValues(lambda v: v[0] / v[1])

pivot_prep_age_rdd = age_avg.map(lambda x: (x[0][0], (x[0][1], x[1])))

grouped_by_movie_age = pivot_prep_age_rdd.groupByKey()

def format_age_ratings(ratings_list):
    groups = {"0-18": "NA", "18-35": "NA", "35-50": "NA", "50+": "NA"}
    for group, avg in ratings_list:
        if group in groups:
            groups[group] = f"{avg:.2f}"
    return f"[0-18: {groups['0-18']}, 18-35: {groups['18-35']}, 35-50: {groups['35-50']}, 50+: {groups['50+']}]"

age_strings_rdd = grouped_by_movie_age.mapValues(format_age_ratings)
final_age_rdd = movies_rdd.join(age_strings_rdd)
formatted_rdd_b4 = final_age_rdd.map(
    lambda x: f"{x[1][0]} - {x[1][1]}"
)

all_results = formatted_rdd_b4.collect()

for line in all_results:
    print(line)

Gladiator (2000) - [0-18: NA, 18-35: 3.44, 35-50: 3.81, 50+: 3.50]
The Terminator (1984) - [0-18: NA, 18-35: 4.17, 35-50: 4.05, 50+: 3.75]
Lawrence of Arabia (1962) - [0-18: NA, 18-35: 3.60, 35-50: 3.29, 50+: 4.50]
Mad Max: Fury Road (2015) - [0-18: NA, 18-35: 3.36, 35-50: 3.64, 50+: NA]
No Country for Old Men (2007) - [0-18: NA, 18-35: 3.81, 35-50: 3.94, 50+: 4.00]
Psycho (1960) - [0-18: NA, 18-35: 4.50, 35-50: 3.50, 50+: NA]
E.T. the Extra-Terrestrial (1982) - [0-18: NA, 18-35: 3.56, 35-50: 3.83, 50+: 3.00]
Fight Club (1999) - [0-18: NA, 18-35: 3.50, 35-50: 3.50, 50+: 3.50]
The Godfather: Part II (1974) - [0-18: NA, 18-35: 3.78, 35-50: 4.25, 50+: NA]
The Lord of the Rings: The Fellowship of the Ring (2001) - [0-18: NA, 18-35: 4.00, 35-50: 3.83, 50+: NA]
The Silence of the Lambs (1991) - [0-18: NA, 18-35: 3.00, 35-50: 3.25, 50+: NA]
Sunset Boulevard (1950) - [0-18: NA, 18-35: 4.17, 35-50: 4.50, 50+: NA]
The Lord of the Rings: The Return of the King (2003) - [0-18: NA, 18-35: 3.83, 35-

Bài 5: Phân tích đánh giá theo nghề nghiệp

In [19]:
user_occ_id_rdd = users_rdd.map(lambda x: (x[0], x[1][2]))

user_rating_rdd = all_ratings_rdd.map(lambda x: (x[0], x[2]))

joined_occ_data = user_occ_id_rdd.join(user_rating_rdd)

occ_rating_kv = joined_occ_data.map(lambda x: (x[1][0], (x[1][1], 1)))

occ_sum_count = occ_rating_kv.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

joined_occ_name = occ_sum_count.join(occupation_rdd)

occ_avg_rdd = joined_occ_name.map(
    lambda x: (x[1][1], (x[1][0][1], x[1][0][0] / x[1][0][1]))
)

formatted_rdd_b5 = occ_avg_rdd.map(
    lambda x: f"{x[0]} - AverageRating: {x[1][1]:.2f} (TotalRatings: {x[1][0]})"
)

all_results = formatted_rdd_b5.collect()

for line in all_results:
    print(line)

Nurse - AverageRating: 3.86 (TotalRatings: 11)
Programmer - AverageRating: 4.25 (TotalRatings: 10)
Salesperson - AverageRating: 3.65 (TotalRatings: 17)
Doctor - AverageRating: 3.69 (TotalRatings: 21)
Accountant - AverageRating: 3.58 (TotalRatings: 6)
Engineer - AverageRating: 3.56 (TotalRatings: 18)
Journalist - AverageRating: 3.85 (TotalRatings: 17)
Designer - AverageRating: 4.00 (TotalRatings: 13)
Teacher - AverageRating: 3.70 (TotalRatings: 5)
Lawyer - AverageRating: 3.65 (TotalRatings: 17)
Artist - AverageRating: 3.73 (TotalRatings: 11)
Consultant - AverageRating: 3.86 (TotalRatings: 14)
Manager - AverageRating: 3.47 (TotalRatings: 16)
Student - AverageRating: 4.00 (TotalRatings: 8)


Bài 6: Phân tích đánh giá theo thời gian

In [20]:
timestamp_rating_rdd = all_ratings_rdd.map(lambda x: (x[3], x[2]))

year_rating_kv = timestamp_rating_rdd.map(
    lambda x: (datetime.datetime.fromtimestamp(x[0]).year, (x[1], 1))
)

year_sum_count = year_rating_kv.reduceByKey(
    lambda a, b: (a[0] + b[0], a[1] + b[1])
)

year_avg = year_sum_count.mapValues(
    lambda v: (v[1], v[0] / v[1])
)

sorted_and_formatted_rdd = year_avg.sortByKey().map(
    lambda x: f"{x[0]} - TotalRatings: {x[1][0]}, AverageRating: {x[1][1]:.2f}"
)

for line in sorted_and_formatted_rdd.collect():
    print(line)

2020 - TotalRatings: 184, AverageRating: 3.75


In [21]:
sc.stop()
print("SparkContext đã dừng.")

SparkContext đã dừng.
